In [20]:
import pandas as pd
from pycaret.regression import *

In [21]:
df = pd.read_csv('Ceny mieszkań.csv', sep=';', encoding='iso-8859-2')
df

,Unnamed: 0,2020 I kwarta¸,2020 II kwarta¸,2020 III kwarta¸,2020 IV kwarta¸,2021 I kwarta¸,2021 II kwarta¸,2021 III kwarta¸,2021 IV kwarta¸,2022 I kwarta¸,2022 II kwarta¸,2022 III kwarta¸,2022 IV kwarta¸,2023 I kwarta¸,2023 II kwarta¸,2023 III kwarta¸,2023 IV kwarta¸,2024 I kwarta¸,2024 II kwarta¸,2024 III kwarta¸
0,GdaÄsk,8 383,8 480,8 882,8 874,9 031,9 071,9 566,10 208,10 239,10 813,10 880,11 526,10 963,10 899,11 629,11 503,12 148,12 243,12 604
1,Gdynia,7 667,7 727,7 996,7 906,8 025,8 465,8 775,8 943,9 245,9 750,10 176,9 777,9 436,9 889,10 051,10 171,10 588,11 524,11 609
2,Sopot,11760,11700,11780,12050,12200,12800,13400,14100,15100,15800,15500,15100,14500,15300,15500,15900,16500,17200,17900
3,Wejherowo,4500,4600,4700,4800,5000,5200,5400,5600,5800,6000,6200,6400,6600,6800,7000,7200,7400,7600,7800
4,Reda,5500,5600,5700,5800,5900,6000,6100,6200,6400,6600,6800,7000,7200,7400,7600,7800,8000,8300,8600
5,Rumia,5500,5600,5700,5800,5900,6000,6100,6200,6400,6600,6800,7000,7200,7400,7600,7800,8000,8300,8600


In [22]:
# Zamiana z szerokiego formatu (kolumny jako kwartały) na długi
df_long = df.melt(id_vars=df.columns[0], var_name='Okres', value_name='Cena')
df_long.columns = ['Miasto', 'Okres', 'Cena']

# Rozdzielenie kolumny "Okres" na rok i kwartał
df_long[['Rok', 'Kwartał']] = df_long['Okres'].str.extract(r'(\d{4})\s+([IVX]+)\s+kwarta')

# Zamiana kwartałów na liczby
map_kwartał = {'I': 1, 'II': 2, 'III': 3, 'IV': 4}
df_long['Kwartał'] = df_long['Kwartał'].map(map_kwartał)

# Czyszczenie kolumny Cena z dziwnych znaków i zamiana na float
df_long['Cena'] = (
    df_long['Cena'].astype(str)
    .str.replace(r'[^\d]', '', regex=True)
    .replace('', pd.NA)
    .astype(float)
)

# Usunięcie braków
df_long.dropna(inplace=True)

# Upewnij się, że Rok to liczba
df_long['Rok'] = df_long['Rok'].astype(int)

df_long.head()

,Miasto,Okres,Cena,Rok,Kwartał
0,GdaÄsk,2020 I kwarta¸,8383.0,2020,1
1,Gdynia,2020 I kwarta¸,7667.0,2020,1
2,Sopot,2020 I kwarta¸,11760.0,2020,1
3,Wejherowo,2020 I kwarta¸,4500.0,2020,1
4,Reda,2020 I kwarta¸,5500.0,2020,1


In [24]:
reg = setup(
    data=df_long,
    target='Cena',
    categorical_features=['Miasto', 'Rok', 'Kwartał'],
    session_id=123,
    verbose=False  
)

In [25]:
best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
dummy,Dummy Regressor,2538.6618,9587792.9000,3010.5376,-0.8313,0.3349,0.3088,0.0160
lightgbm,Light Gradient Boosting Machine,2560.3942,9770984.6628,3037.8464,-0.8859,0.3380,0.3119,0.0540


Processing:   0%|          | 0/77 [00:00<?, ?it/s]